# Imports

In [1]:
import pysweep
import qcodes as qc
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
import pysweep.convenience as conv
import random
import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from pysweep.databackends.base import DataParameter
from pytopo.qctools.dataset2 import select_experiment
from qcodes.dataset.measurements import Measurement

from qcodes import load_by_id
import qcodes as qc
import qcodes.dataset.plotting
import importlib

C:\Anaconda3\envs\qcodes-env2\lib\site-packages\qcodes\dataset\sqlite_base.py:51: UserWarning: The module `qcodes.dataset.sqlite_base` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.sqlite_base` is deprecated.\n'


In [3]:
station = qc.Station()
pysweep.STATION = station
qc.config.core.db_location = r'testdata.db'
qc.initialise_database()

# Define a few parameters with physics

In [4]:
bias = qc.instrument.parameter.ManualParameter("bias", units="V")
T1 = qc.instrument.parameter.ManualParameter("T1", units="V")
T2 = qc.instrument.parameter.ManualParameter("T2", units="V")

bias(0)
T1(0)
T2(0)

def pinchoff(v, rs=10e3, l=2e-6, muc=3*60e-18, vth=-1):
    if v>vth:
        return (rs+l**2/(muc*(v-vth)))
    else:
        return 1e11

def get_current():
    u = bias()
    r = pinchoff(T1())+pinchoff(T2())
    return u/r
current = qc.instrument.parameter.Parameter("current", units="A", get_cmd=get_current)



# Define the functions that define the measurement

We need to define what happens at the start of a measurement, what must happen at the end of a measurement and finally what a measurement entails

In [5]:
def init_measurement(d):
    bias(1e-3)

def end_measurement(d):
    bias(0)

Any measurement that pysweep does is encapsulated as a MeasurementFucntion class. It contains a function that returns the measurement results as a list, as well as a list containing the metadata for every return value, this list is called the `ParamStruct`

In [6]:
@MakeMeasurementFunction([pysweep.DataParameter(name='current', unit='A')])
def measure(d):
    return [current()]

In [7]:
type(measure)

pysweep.core.measurementfunctions.MeasurementFunction

In [8]:
measure.paramstruct

[<class 'pysweep.databackends.base.DataParameter'> current A numeric False]

lets also define a function the measures the hypothetical temperature of our fridge
Because the explicit definition opf the DataParameter is overly verbose, we introduce the shorthand:

If an element `param` of the datastruct is not of type `DataParameter`, it will try to construct a dataparameters automatically as
`DataParameter(*param)`
as such we can define a measurementfunction as

In [9]:
@MakeMeasurementFunction([['Temperature', 'mK']])
def measure_temperature(d):
    return [random.randint(100, 300)/10]

# Simple measurements - Simple Sweep objects

Since we now have the functions that will be execute to measure something, we can now look at the code that will perform the sweeping

In pysweep, sweeps are performed with `SweepObjects` which consist of 4 elements
* set_function
* unit
* label
* point_function

For now, let us ignore the details and see how they can be constructed:

`pysweep.sweep_object(T1, np.linspace(0,-1.2,51))`

This encodes the idea of sweeping the `T1` gate from 0 to -1.2 in 51 steps (`np.linspace(0,-1.2,51)`)

With this extra component, we have enouigh to start performing measurements. This paragraph contains some measurements that corresponds to the easiest cases of measurements. The cases cover 90% of all measurements that will run, but don't show the full power of pysweep

In [ ]:
exp = select_experiment('pinchoff', 'test_sample')
meas = Measurement(exp, station)
r = pysweep.sweep(init_measurement, end_measurement, measure, 
              pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
              databackend = pysweep.databackends.qcodes.DataBackend(meas)
            )

In [ ]:
qcodes.dataset.plotting.plot_by_id(r.datasaver.run_id)

Adding another dimension is trivial, where we follow the rule that the inner most loop is at the top, closest to the `MeasurementFunction`

In [ ]:
exp = select_experiment('pinchoff', 'test_sample')
meas = Measurement(exp, station)
r = pysweep.sweep(init_measurement, end_measurement, measure, 
              pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
              pysweep.sweep_object(T2, np.linspace(0,-1.2,51)), 
              databackend=pysweep.databackends.qcodes.DataBackend(meas)
            )

In [ ]:
qcodes.dataset.plotting.plot_by_id(r.datasaver.run_id)

# Extended examples

Here are a few examples of measurements that are still relatively simples, but with a few more bells and whistles

Say that we want to measure the fridge temperature in addition to the current. This can be done by creating the combined measurement object `measure+measure_temperature` and hence we simply execute

In [ ]:
exp = select_experiment('pinchoff', 'test_sample')
meas = Measurement(exp, station)
r = pysweep.sweep(init_measurement, end_measurement, measure+measure_temperature, 
              pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
              pysweep.sweep_object(T2, np.linspace(0,-1.2,51)), 
              databackend=pysweep.databackends.qcodes.DataBackend(meas)
            )

In [ ]:
qcodes.dataset.plotting.plot_by_id(r.datasaver.run_id)

But often we only want to measure temperature every now and then, so let's assume that we want to measure the fridge tmeperature only after every time we set the T2 parameter

In [ ]:
exp = select_experiment('pinchoff', 'test_sample')
meas = Measurement(exp, station)
r = pysweep.sweep(init_measurement, end_measurement, measure, 
              pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
              conv.add_function(pysweep.sweep_object(T2, np.linspace(0,-1.2,51)), measure_temperature), 
              databackend = pysweep.databackends.qcodes.DataBackend(meas)
            )

In [ ]:
qcodes.dataset.plotting.plot_dataset(r.datasaver.dataset)

# Hardware sweep

Often, hardware is used to sweep one of the parameters instead of software. Pysweep deals with these situations natively. Let's assume that we use a VNA to measure a conductance resonator in the sample, we first define the code that measures the VNA. In this case we write a mock version of this to simulate the result

In [10]:
@MakeMeasurementFunction([['frequency', 'Herz', 'array', True], ['S11', 'dB', 'array']])
def measure_vna(d):
    freqs = np.linspace(440e6, 460e6, 201)
    omega = 2*np.pi*freqs
    l = 420e-9
    c = 0.3e-12
    r = pinchoff(T1())+pinchoff(T2())
    z = 1j*omega*l+1/(1/r + 1j*omega*c)
    z_0 = 50
    Gamma = (z - z_0)/(z + z_0)
    
    return [freqs, 10 + 20 * np.log10(np.abs(Gamma))-70]

Where we make explicit in the parameter definition that frequency and S11 are parameters of type array and that frequency is an independent parameter

When running the measurement, ensure to reopen T2 to measure the pinchoff along T1

In [ ]:
exp = select_experiment('rf_pinchoff', 'test_sample')
meas = Measurement(exp, station)
T2(0)
r = pysweep.sweep(init_measurement, end_measurement, measure_vna, 
              pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
              databackend = pysweep.databackends.qcodes.DataBackend(meas)
            )

In [ ]:
qcodes.dataset.plotting.plot_dataset(r.datasaver.dataset)

# A note on parameter dependencies

To figure out which parameters depend on which parameters, pysweep uses a simple rule: every parameter depends on ALL independent parameters that occur earlier in the `paramstruct`

looking at the paramstruct of the previous measurement:

In [ ]:
r.columns

This assumption is usually correct but can lead to problems, for example in the next measurement which does not work
Because of the preceding rule, pysweep expects in the following piece of code that `current` will depend on `frequency`, which is of course not the case.

In [ ]:
exp = select_experiment('rf_pinchoff', 'test_sample')
meas = Measurement(exp, station)
T2(0)
try:
    r = pysweep.sweep(init_measurement, end_measurement, measure_vna+measure, 
                  pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
                  databackend = pysweep.databackends.qcodes.DataBackend(meas)
                )
except ValueError as e:
    print(e)

### Simple solution

Switching around the two measurement functions solves this issue:

In [ ]:
exp = select_experiment('rf_pinchoff', 'test_sample')
meas = Measurement(exp, station)
T2(0)
r = pysweep.sweep(init_measurement, end_measurement, measure+measure_vna, 
                  pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
                  databackend = pysweep.databackends.qcodes.DataBackend(meas)
                )

In [ ]:
qcodes.dataset.plotting.plot_dataset(r.datasaver.dataset)

### Advanced solution

In some cases it might be the case that both orders will not work. The usecase might be two incompatible hardware sweeps bundeled together.


In this case the issue can also be resolved bu explicitely telling pysweep that frequency and S11 are together and do not interfere with any other parameter.
Because the implicit syntax for the dataparameters gets confusing, I'll use the explicit notation here

In [11]:
@MakeMeasurementFunction([pysweep.DataParameter(name='frequency_2',
                                              unit='Hz',
                                               paramtype='array',
                                               independent=2  # yes independent, but pysweep will not recognize it as such
                                            ),
                         pysweep.DataParameter(name='S11_2', 
                                              unit='dB',
                                              paramtype='array',
                                              extra_dependencies=['frequency_2']  # explicitely tell that this parameter depends on frequency_2
                                              )])
def measure_vna2(d):
    freqs = np.linspace(440e6, 460e6, 11)
    omega = 2*np.pi*freqs
    l = 420e-9
    c = 0.3e-12
    r = pinchoff(T1())+pinchoff(T2())
    z = 1j*omega*l+1/(1/r + 1j*omega*c)
    z_0 = 50
    Gamma = (z - z_0)/(z + z_0)
    
    return [freqs, 10 + 20 * np.log10(np.abs(Gamma))-70]

In [ ]:
exp = select_experiment('rf_pinchoff', 'test_sample')
meas = Measurement(exp, station)
T2(0)
r = pysweep.sweep(init_measurement, end_measurement, measure_vna2+measure_vna, 
                  pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
                  databackend = pysweep.databackends.qcodes.DataBackend(meas)
                )

In [ ]:
qcodes.dataset.plotting.plot_dataset(r.datasaver.dataset)

Pysweep (with the qcodes backend) only allows one parameter with the same name per measurement, to avoid overwriting data. This is why we renamed the parameters initially

In [12]:
@MakeMeasurementFunction([pysweep.DataParameter(name='frequency',
                                              unit='Hz',
                                               paramtype='array',
                                               independent=2  # yes independent, but pysweep will not recognize it as such
                                            ),
                         pysweep.DataParameter(name='S11', 
                                              unit='dB',
                                              paramtype='array',
                                              extra_dependencies=['frequency']  # explicitely tell that this parameter depends on frequency_2
                                              )])
def measure_vna2(d):
    freqs = np.linspace(440e6, 460e6, 11)
    omega = 2*np.pi*freqs
    l = 420e-9
    c = 0.3e-12
    r = pinchoff(T1())+pinchoff(T2())
    z = 1j*omega*l+1/(1/r + 1j*omega*c)
    z_0 = 50
    Gamma = (z - z_0)/(z + z_0)
    
    return [freqs, 10 + 20 * np.log10(np.abs(Gamma))-70]

In [13]:
exp = select_experiment('rf_pinchoff', 'test_sample')
meas = Measurement(exp, station)
T2(0)
r = pysweep.sweep(init_measurement, end_measurement, measure_vna2+measure_vna, 
                  pysweep.sweep_object(T1, np.linspace(0,-1.2,51)), 
                  databackend = pysweep.databackends.qcodes.DataBackend(meas)
                )

None3 , postponing registration
None2 , postponing registration


ValueError: Parameter name frequency occurs multiple times in paramstruct

# Adaptive sweep - under construction

In [ ]:
@MakeMeasurementFunction([['timestamp', 's']])
def smart_point_function(dict_waterfall):
    timestamp = int(time.time())
    points = [int(c) for c in str(timestamp)]
    return points, [timestamp]

In [ ]:
db = pysweep.databackends.debug.DebugDataBackend()
pysweep.sweep(lambda d:{}, lambda d:{}, measure, 
              pysweep.sweep_object(a, [0,1]),
              conv.add_function(pysweep.sweep_object(b, [3,4]), measure_temperature),
              pysweep.sweep_object(c, smart_point_function, dataparameter=np.linspace(0,9,10)), databackend=db
             )

# Real life example - under construction

In [ ]:
%%enqueue single_shot_readout
exp = select_experiment('blobs', SAMPLE)
meas = Measurement(exp, station)
singleshot_dataextractor.setup_acquisition(1024, 1000, 100)
#4.77946e9
parameter_list = [[station.TI.NW_bias, 0e-3], [station.HT.frequency, 4779770000], [station.HT.RF.power, -20.25]]

pysweep.sweep(sf.set_parameters_after(init_rf, parameter_list), end_rf, 
              measure_singleshot_rf,
              pysweep.sweep_object(station.TI.SP4, snr_calibrate_plunger(pysweep.sweep_object(station.TI.SP4, np.linspace(0., 0.4, 101))), dataparameter=[0,1]),
              pysweep.convenience.add_function(pysweep.sweep_object(station.TI.T5, np.linspace(-1, -1.35, 101)), csc),
              databackend = pysweep.databackends.qcodes.DataBackend(meas)
             )
rigol.ch1_output_enabled(False)

# Jagged Data Structures - under construction

In [ ]:
?DataParameter

In [ ]:
@MakeMeasurementFunction([DataParameter('time', 's', 'array', 2),
                         DataParameter('first_gate', 'V', 'array', 2),
                         DataParameter('second_gate', 'V', 'array', 2),
                         DataParameter('first_gate_result', 'I', 'array', extra_dependencies=['time', 'first_gate']),
                         DataParameter('time', 's', 'array', 2, duplicate=True),
                         DataParameter('second_gate_result', 'I', 'array', extra_dependencies=['time', 'second_gate'])])
def some_arbitrary_data(d):
    return [np.linspace(0,1, 11), np.linspace(0,0.1, 11), np.linspace(0,-0.1, 21), np.linspace(0,0.2, 11), np.linspace(1,2, 21), np.linspace(0,-0.2, 21)]

In [ ]:
importlib.reload(pysweep.databackends.qcodes)

In [ ]:
exp = select_experiment('jagged_data', 'test')
meas = Measurement(exp, station)

pysweep.sweep(init_measurement, end_measurement, 
              some_arbitrary_data,
              pysweep.sweep_object(T1, np.linspace(0,1,2)),
              databackend = pysweep.databackends.qcodes.CutDataBackend(meas)
             )

In [ ]:
raw_data = load_by_id(12)
data = raw_data.get_parameter_data()

In [ ]:
raw_data

In [ ]:
data['first_gate_result']['first_gate_result']

In [ ]:
plt.plot(data['first_gate_result']['first_gate'][0], data['first_gate_result']['first_gate_result'][0], '.')
plt.show()
plt.plot(data['second_gate_result']['second_gate'][0], data['second_gate_result']['second_gate_result'][0], '.')
plt.show()
plt.plot(data['first_gate_result']['time'][0], data['first_gate_result']['first_gate_result'][0], '.')
plt.plot(data['second_gate_result']['time'][0], data['second_gate_result']['second_gate_result'][0], '.')
plt.xlabel('time')
plt.ylabel('gate_result')
plt.show()

In [ ]:
from plottr.data.datadict import datadict_to_meshgrid
from plottr.data.qcodes_dataset import ds_to_datadict

In [ ]:
def load(runid):
    return datadict_to_meshgrid(ds_to_datadict(load_by_id(runid)))

In [ ]:
ds = ds_to_datadict(load_by_id(12))

In [ ]:
from plottr.node.data_selector import DataSelector

In [ ]:
dataselector = DataSelector('dataselector')

In [ ]:
ds

In [ ]:
ds2 = ds.extract(['first_gate_result'])

In [ ]:
ds3 = datadict_to_meshgrid(ds2)
ds3.validate()

In [ ]:
ds3.shape()

In [ ]:
ds3.structure(), ds3.shapes()

In [ ]:
ds3.sanitize()